In [7]:
from dlomix.data.fragment_ion_intensity import FragmentIonIntensityDataset
from dlomix.constants import PTMS_ALPHABET
import tensorflow as tf

In [12]:
ds = FragmentIonIntensityDataset(
    '/cmnfs/proj/bmpc_dlomix/datasets/parquet/noptm_baseline_small_train.parquet',
    inference_only=True,
    alphabet=PTMS_ALPHABET,
    encoding_scheme='naive-mods',
    model_features=["precursor_charge_onehot", "collision_energy_aligned_normed","method_nbr"]
)

Generating train split: 8000 examples [00:00, 48347.93 examples/s]
/cmnfs/home/f.kapitza/dlomix/src/dlomix/data/dataset.py:364: UserWarning: 
                This is a inference only dataset! You can only make predictions with this dataset! Attempting to
                train a model with this dataset will result in an error!
                
  warnings.warn(
Casting the dataset: 100%|██████████| 7988/7988 [00:01<00:00, 5905.14 examples/s]


In [13]:
ds.hf_dataset

DatasetDict({
    inference: Dataset({
        features: ['modified_sequence', 'precursor_charge_onehot', 'collision_energy_aligned_normed', 'method_nbr', '_parsed_sequence', '_n_term_mods', '_c_term_mods'],
        num_rows: 7988
    })
})

In [14]:
batch = next(iter(ds.tensor_inference_data))

In [15]:
batch

{'modified_sequence': <tf.Tensor: shape=(64, 30), dtype=int64, numpy=
 array([[21, 17,  6, ...,  0,  0,  0],
        [21,  1,  7, ...,  0,  0,  0],
        [21, 12,  4, ...,  0,  0,  0],
        ...,
        [21,  5, 10, ...,  0,  0,  0],
        [21,  5,  3, ...,  0,  0,  0],
        [21,  6, 12, ...,  0,  0,  0]])>,
 'precursor_charge_onehot': <tf.Tensor: shape=(64, 6), dtype=float32, numpy=
 array([[0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 

In [10]:
from dlomix.losses import masked_spectral_distance
model = tf.keras.models.load_model('/cmnfs/proj/bmpc_dlomix/models/baseline_models/noptm_baseline_full_bs1024/4bc7bc69-bbf4-4366-90fc-474b1946c588.keras')

In [17]:
preds = model.predict(batch)

2/2 [==============================] - 0s 83ms/step


In [22]:
max(preds[0])

0.22409272